In [61]:
import json
import pandas as pd
import sqlite3
import spacy 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Load data from newscatcher

In [62]:
def add_unique_id(df):
    """
    Add a unique ID column to the DataFrame.
    """
    df['unique_id'] = range(1, len(df) + 1)
    return df

def load_data(filename,db_name,table_name):
    with open(filename, 'r') as f:
        json_data = json.load(f)
    df = pd.DataFrame(json_data['articles'])
    
    # Step 2: Add Unique ID
    df = add_unique_id(df)
    
    # Step 3: Create SQLite DB
    conn = sqlite3.connect(db_name)
    
    # Step 4: Create Table and Insert Data
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Successfully loaded data into {db_name}, table name: {table_name}")
    conn.close()
    return 

def mass_load(filenames,db_name,table_name):
    for name in filenames:
        load_data(name,db_name,table_name) 
    return

## Process data 

In [69]:
## TODO Locations 
### TODO Add col names vader_neg, vader_neu, vader_pos, vader_pol
### TODO Database schema, multivar process 
from bertopic import BERTopic

analyzer = SentimentIntensityAnalyzer()
nlp = spacy.load("en_core_web_trf",exclude=['tagger', 'attribute_ruler', 'lemmatizer'])

def flatten_concatenation(matrix):
     flat_list = []
     for row in matrix:
         flat_list += row
     return flat_list

def process_text(data, var):
    text_data = [k[var] for k in data]
    uuid = [k[var] for k in data]
    result = []
    # Sentiment  
    for k in data:
        result.append(analyzer.polarity_scores(k[var]).values())
    
    # Basic NLP
    for k in data:
        doc = nlp(k[var])
        for ent in doc.ents:
            result.append(dict([('ent_text',ent.text),('ent_type',ent.label_)]))
            
    # BERTopic 
    # topic_model = BERTopic(embedding_model=nlp)
    # topics, probs = topic_model.fit_transform(text_data)
    # fig = topic_model.visualize_topics()
    # fig.show()
    
    return result

# Testing
## Load and create DB 

In [70]:
## SQL Skeleton "SELECT cols FROM table WHERE conditions"

con = sqlite3.connect(db_name)
cur = con.cursor()
cur.execute('SELECT * FROM articles_table')
data2 = cur.fetchall()
df2 = pd.read_sql_query("SELECT title,topic,summary,excerpt,published_date,clean_url,topic from articles_table", con)
test_result = process_text(data, 2)

In [72]:
df2.head()

,title,topic,summary,excerpt,published_date,clean_url,topic
0,Ana Navarro Says Illegal Migrants Need To Be ‘...,politics,'The View' co-host Ana Navarro called for ille...,"""The View"" co-host Ana Navarro called for ille...",2023-09-08 18:10:21,dailycaller.com,politics
1,Illegal Senegalese migrants among thousands re...,news,By\n\nMigrants from Senegal are among the thou...,Migrants from Senegal are among the thousands ...,2023-09-14 10:23:45,dailymail.co.uk,news
2,EXCLUSIVE PHOTOS: 300 Migrants Stroll Across B...,news,"LUKEVILLE, Arizona — A single group of nearly ...",A group of nearly 300 migrants walked through ...,2023-09-04 19:07:40,breitbart.com,news
3,Tiny Italian island coping with thousands of m...,news,A migrant reception center in Italy's southern...,A migrant reception center in Italy's southern...,2023-09-14 05:31:20,breitbart.com,news
4,Tiny Italian island coping with thousands of m...,news,A migrant reception center in Italy's southern...,A migrant reception center in Italy's southern...,2023-09-14 11:52:51,independent.co.uk,news


In [88]:
## TODO Locations 
### TODO Add col names vader_neg, vader_neu, vader_pos, vader_pol
### TODO Database schema, multivar process 
from bertopic import BERTopic

analyzer = SentimentIntensityAnalyzer()
nlp = spacy.load("en_core_web_trf",exclude=['tagger', 'attribute_ruler', 'lemmatizer'])

def flatten_concatenation(matrix):
     flat_list = []
     for row in matrix:
         flat_list += row
     return flat_list

def process_text(data, var):
    text_data = [k[var] for k in data]
    uuid = [k[0] for k in data]
    result = []
    result.append(uuid)
    # Sentiment  
    for k in data:
        result.append(analyzer.polarity_scores(k[var]))
    
    # Basic NLP
    for k in data:
        doc = nlp(k[var])
        for ent in doc.ents:
            result.append(dict([('ent_text',ent.text),('ent_type',ent.label_)]))
            
    # BERTopic 
    # topic_model = BERTopic(embedding_model=nlp)
    # topics, probs = topic_model.fit_transform(text_data)
    # fig = topic_model.visualize_topics()
    # fig.show()
    
    return result
# Testing
## Parameters 

In [96]:
df2.head()

,_id,title,topic,summary,excerpt,published_date,clean_url,topic
0,fa68ea16830c2cc8b3d985dbafb27d53,Ana Navarro Says Illegal Migrants Need To Be ‘...,politics,'The View' co-host Ana Navarro called for ille...,"""The View"" co-host Ana Navarro called for ille...",2023-09-08 18:10:21,dailycaller.com,politics
1,7e7755289f5e940dd27a302f539fc031,Illegal Senegalese migrants among thousands re...,news,By\n\nMigrants from Senegal are among the thou...,Migrants from Senegal are among the thousands ...,2023-09-14 10:23:45,dailymail.co.uk,news
2,e43c0591ce65896c0f2eeb34538db5b2,EXCLUSIVE PHOTOS: 300 Migrants Stroll Across B...,news,"LUKEVILLE, Arizona — A single group of nearly ...",A group of nearly 300 migrants walked through ...,2023-09-04 19:07:40,breitbart.com,news
3,86c1ae233098d27344d57a682f591f05,Tiny Italian island coping with thousands of m...,news,A migrant reception center in Italy's southern...,A migrant reception center in Italy's southern...,2023-09-14 05:31:20,breitbart.com,news
4,9461b77fb30ec500361664c447d17645,Tiny Italian island coping with thousands of m...,news,A migrant reception center in Italy's southern...,A migrant reception center in Italy's southern...,2023-09-14 11:52:51,independent.co.uk,news


In [97]:
uuid = [k[0] for k in data]

In [98]:
uuid[0]

"Ana Navarro Says Illegal Migrants Need To Be ‘Resettled Elsewhere'"

In [99]:
## SQL Skeleton "SELECT cols FROM table WHERE conditions"

con = sqlite3.connect(db_name)
cur = con.cursor()
cur.execute("SELECT _id, title,topic,summary,excerpt,published_date,clean_url,topic from articles_table")
data2 = cur.fetchall()
df2 = pd.read_sql_query("SELECT _id, title,topic,summary,excerpt,published_date,clean_url,topic from articles_table", con)
test_result = process_text(data2, 2)

In [100]:
test_result

[['fa68ea16830c2cc8b3d985dbafb27d53',
  '7e7755289f5e940dd27a302f539fc031',
  'e43c0591ce65896c0f2eeb34538db5b2',
  '86c1ae233098d27344d57a682f591f05',
  '9461b77fb30ec500361664c447d17645',
  '4ca6a0a4e2f4e0a31d7ba52b46fe5e94',
  '7789ec4fc05f365a192e7237a24355c4',
  'ba3a542e3969033fcb852bf46af4e637',
  'fffb2aceb1c8543d40507e3e721b5ca2',
  'cc2284e78c72dc5b4d7a58c1e59686cc',
  '422bd84c4ed478616c60342ea7ce95a6',
  '3164b3c02a5f608b06476c32883d0263',
  '0a0b69b2e5a68f60ed9345e2ab12e36e',
  'cb7caa4011a7fa32e02730969a2bb1d9',
  'cc75a1100948a84ccfe5cb5deb993355',
  '3d1563ee2b79e74a792c479f3c914afb',
  '14844d9b767d030b2b35fe5ea19cabe1',
  '1d770886ae9343a26f5f6bfef169904d',
  'd92cdff890f63cbf8d12edffd3981787',
  '296dc00f0d888c4ea7553ce5743cf7e2'],
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, '